In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# Define the computation model of a camera
class Camera:    
    def __init__(self, focal1, focal2, w, h, pose=None):
        """
        :param pose: (4, 4).
        """
        self.focal1 = focal1
        self.focal2 = focal2
        self.w = w
        self.h = h
        self.pose = pose
        
    def project(self, points, pose=None):
        """
        :param points: (N, 3)
        :param pose: (4, 4).
        """
        if pose is None:
            pose = self.pose
        R, t = pose[:3, :3], pose[:3, 3]
        uvd = np.einsum('ij,nj->ni', R, points) + t
        
        u, v = uvd[:, 0] / uvd[:, 2], uvd[:, 1] / uvd[:, 2]
        u = u * self.focal1 + self.w
        v = v * self.focal2 + self.h
        uv = np.stack([u, v], 1)
        return uv

In [3]:
# Create the camera
camera = Camera(focal1=420.50671,
                focal2=420.61093,
                w=355.20831,
                h=250.33679)

# Load camera poses
poses = np.load('poses.npy')

In [4]:
# Create the pyramid
def create_pyramid():
    verts = np.array(
        [[0, 0, 0],
         [0.16, 0, 0],
         [0.16, 0.16, 0],
         [0, 0.16, 0],
         [0.08, 0.08, -0.12]]
    )
    edges = [[0, 1], [1, 2], [2, 3], [3, 0],
             [0, 4], [1, 4], [2, 4], [3, 4]]
    return verts, edges

verts, edges = create_pyramid()

# Draw AR pyramid on several images
fig, axes = plt.subplots(1, 3, figsize=(20, 3))

for idx, frame_idx in enumerate([5, 15, 25]):
    img_file = 'images/img_%04d.jpg'%(frame_idx)
    img = cv2.imread(img_file)
    
    pose = poses[frame_idx]
    verts_uv = camera.project(verts, pose)
    
    for edge in edges:
        cv2.line(img, (int(verts_uv[edge[0], 0]), int(verts_uv[edge[0], 1])),
                 (int(verts_uv[edge[1], 0]), int(verts_uv[edge[1], 1])), (255, 0, 0), 3)
    
    axes[idx].imshow(img)